# Develops the Calculation Model for the Online Calculator

In [1]:
import sys
from math import nan, isnan, inf
import numbers
from importlib import reload
import inspect
import pandas as pd
import numpy as np
# import matplotlib pyplot commands
from matplotlib.pyplot import *
from IPython.display import Image, Markdown
from qgrid import show_grid as sh
 
# Show Plots in the Notebook
%matplotlib inline
 
# 'style' the plot like fivethirtyeight.com website
style.use('bmh')

In [2]:
rcParams['figure.figsize']= (10, 6)   # set Chart Size
rcParams['font.size'] = 14            # set Font size in Chart

In [3]:
# Access the directory where some utility modules are located in the
# actual heat pump calculator.
sys.path.insert(0, '../../heat-pump-calc/heatpump/')

In [4]:
import library as lib
reload(lib)

<module 'library' from '../../heat-pump-calc/heatpump/library.py'>

In [28]:
sh(lib.df_city[['Name']])

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

## Object that Calculates Monthly Electric Utility Costs 

In [6]:
def chg_nonnum(val, sub_val):
    """Changes a nan or anything that is not a number to 'sub_val'.  
    Otherwise returns val.
    """
    if isinstance(val, numbers.Number):
        if isnan(val):
            return sub_val
        else:
            return val
    else:
        return sub_val


class ElecCostCalc:
    """Class to calculate monthly electric cost given a block rate structure possibly including
    PCE.
    """

    def __init__(
        self,
        utility, 
        sales_tax=0.0,      
        pce_limit=500.0,
        ):
        """Constructor parameters:
        utility:            The electric utility object from the library module.
        sales_tax:          sales tax to apply to total electric bill.
        pce_limit:          maximum number kWh that PCE can apply to. 
                                use math.nan or math.inf to indicate no limit.
        """
        # save some of the variables for use in other methods
        # add Sales Tax to any rate elements
        self.utility = utility
        self.sales_tax = sales_tax
        
        # Make a set of blocks that:
        #   * have the quantity of kWh in the block instead of the block upper limit kWh
        #   * include a block to accomodate a PCE limit
        #   * account for the effects of PCE

        # Convert PCE limit to infinite if no limit
        pce_limit = chg_nonnum(pce_limit, inf)

        if pce_limit > 0:
            # get the correct PCE rate and convert anything not a number to 0.
            pce_adj = chg_nonnum(self.utility.PCE, 0.0)
        else:
            pce_adj = 0.0

        # Flag to indicate whether a PCE block has been added.  If PCE is zero
        # no need to add a block.        
        pce_block_added = (pce_adj==0.0)  

        # Make a new set of blocks that includes the PCE limit as a new block
        # and the set is only as long as it needs to be.
        selected_blocks = self.utility.Blocks
        done = False
        temp_blocks = []
        for max_kwh, rate in selected_blocks:
            b_kwh = chg_nonnum(max_kwh, inf)
            done = (b_kwh == inf)

            # Determine whether it is time to insert the PCE block or not.
            # Test is whether block quantity exceeds the PCE limit.
            if not pce_block_added and (b_kwh > pce_limit):
                # Need to insert the PCE block now.
                temp_blocks.append( (pce_limit, rate) )
                pce_block_added = True          # PCE block has now been added.

            # address the case where the PCE limit matches the upper block
            if pce_limit == b_kwh:
                pce_block_added = True

            # add the block
            temp_blocks.append( (b_kwh, rate))

            if done:
                break

        # Include the PCE adjustment, and sales tax.
        # Also, convert the block quantities so that they are the quantity of 
        # kWh in the block instead of the upper limit of the block.
        prev_upper = 0.0
        self._blocks = []    # final block list
        for max_kwh, rate in temp_blocks:
            if max_kwh <= pce_limit:
                rate -= pce_adj
            rate *= (1. + sales_tax)
            self._blocks.append( (max_kwh - prev_upper, rate) )
            prev_upper = max_kwh

    def monthly_cost(self, kwh_energy, kw_demand=0.0):
        """Returns the total electric cost for a month, given energy usage of
        'kwh_energy' and peak demand of 'kw_demand'.
        """
        # customer charge
        cost = chg_nonnum(self.utility.CustomerChg, 0.0) * (1. + self.sales_tax)

        # demand charge
        cost += kw_demand * chg_nonnum(self.utility.DemandCharge, 0.0) * (1. + self.sales_tax)

        remaining_kwh = kwh_energy
        for b_kwh, b_rate in self._blocks:
            kwh_in_block = min(remaining_kwh, b_kwh)
            cost += kwh_in_block * b_rate
            remaining_kwh -= kwh_in_block
            if remaining_kwh < 0.1:         # 0.1 kWh in case of rounding issues
                break
        
        return cost

    def final_blocks(self):
        """Debug method to return underlying rate blocks
        """
        return self._blocks

In [7]:
util_id = 383
utility = lib.util_from_id(util_id)
ec = ElecCostCalc(utility, sales_tax=0.0, pce_limit=500.0)
print(ec.utility.Blocks)
ec.final_blocks()

[(100.0, 0.817300021299161), (400.0, 0.742400020011701), (500.0, 0.667500018724241), (1000.0, 0.592500000842847), (nan, 0.562600016244688)]


[(100.0, 0.44630002940539304),
 (300.0, 0.37140002811793305),
 (100.0, 0.29650002683047305),
 (500.0, 0.592500000842847),
 (inf, 0.562600016244688)]

In [8]:
ec.monthly_cost(401)

156.34651140274968

In [9]:
util_id = 1
utility = lib.util_from_id(util_id)
ec = ElecCostCalc(utility, sales_tax=0.0, pce_limit=500.0)
print(ec.final_blocks())
utility

[(inf, 0.18647000112105194)]


ID                                                              1
Name                                Chugach Electric- Residential
Active                                                       True
Type                                                            1
IsCommercial                                                False
ChargesRCC                                                   True
PCE                                                             0
CO2                                                           1.1
CustomerChg                                                     8
DemandCharge                                                  NaN
NameShort                                                  Chugac
Blocks          [(nan, 0.18647000112105194), (nan, nan), (nan,...
Name: 1, dtype: object

In [10]:
ec.monthly_cost(0, 1)

8.0

In [11]:
custom_ut = utility.copy()
custom_ut['DemandCharge'] = 10.0
ec = ElecCostCalc(custom_ut, sales_tax=0.0, pce_limit=500.0)
ec.monthly_cost(0, 1.0)

18.0

## Home Space Heating Simulation Model

In [12]:
class HomeEnergyModel(object):
    
    def __init__(self,
                 city_id,
                 hp_model_id,
                 exist_heat_fuel_id,
                 exist_heat_effic,
                 exist_kwh_per_mmbtu,     # Boiler: 5.5, Toyo: 3, Oil Furnace: 8.75 - 15
                 exist_is_point_source,
                 co2_lbs_per_kwh,
                 low_temp_cutoff,
                 garage_stall_count,
                 garage_heated_by_hp,
                 bldg_floor_area,
                 indoor_heat_setpoint,
                 insul_level,             # 1 - 2x4, 2 - 2x6, 3 - better than 2x6 Walls
                 pct_exposed_to_hp,
                 doors_open_to_adjacent,
                 low_temp_acceptance,     # 1 - no temp drop in back rooms, 2 - 4 deg F cooler, 10 deg F cooler

                 # The inputs below are not user inputs, they control the 
                 # calculation process. They are give default values.
                 hp_only=False,           # when using heat pump, it's the only heat source for home
                 no_heat_pump_use=False,  # If True, models existing heating system alone.
                 ua_true_up=1.0,          # used to true up calculaion to actual fuel use
                 **kwargs
                ):
        
        # Store all of these input parameters as object attributes
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        for arg in args[1:]:
            setattr(self, arg, values[arg])

    def __repr__(self):
        """Returns a string with all the object attributes shown.  Text is truncated
        at 1,000 characters for attributes with long representations.
        """
        s = ''
        for attr in self.__dict__:
            val = repr(self.__dict__[attr])[:1000]
            if len(val)>70:
                s+=f'\n{attr}:\n{val}\n\n'
            else:
                s += f'{attr}: {val}\n'
        return s
    
    # ------- Data needed for calculation of COP vs. Temperature
    
    # Piecewise linear COP vs. outdoor temperature.
    COP_vs_TEMP = (
        (-20.0, 1.1),
        (0.0, 2.0),
        (10.0, 2.2),
        (15.0, 2.3),
        (20.0, 2.5),
        (25.0, 2.7),
        (30.0, 2.8),
        (40.0, 3.0),
        (50.0, 3.2)
    )

    # convert to separate lists of temperatures and COPs
    TEMPS_FIT, COPS_FIT = tuple(zip(*COP_vs_TEMP))

    # The HSPF value that this curve is associated with
    BASE_HSPF = 13.3
    
    # ---------------- Main Calculation Method --------------------
    
    def calculate(self):
        """Main calculation routine that models the home and determines
        loads and fuel use by hour.  Also calculates summary results.
        """
        # Set up some shortcut variables to shorten calculation code
        s = self
        
        # ------ Create object variables from the input IDs
        s.city = lib.city_from_id(s.city_id)
        s.hp_model = lib.heat_pump_from_id(s.hp_model_id)
        s.exist_heat_fuel = lib.fuel_from_id(s.exist_heat_fuel_id)
        
        # ------ Make a DataFrame with hourly input information
        # Do as much processing at this level using array operations, as
        # opposed to processing within the hourly loop further below.
        
        df_tmy = lib.tmy_from_id(self.city.TMYid)
        s.df_hourly = df_tmy[['db_temp']].copy()
        # and now make a shorthand variable for this DataFrame
        dfh = s.df_hourly
        dfh['day_of_year'] = dfh.index.dayofyear
        dfh['month'] = dfh.index.month

        # Determine days that the heat pump is running.  Look at the minimum
        # temperature for the day, and ensure that it is above the low 
        # temperature cutoff.
        hp_is_running = lambda x: (x.min() > self.low_temp_cutoff)
        dfh['running'] = dfh.groupby('day_of_year')['db_temp'].transform(hp_is_running)

        # Determine a heat pump COP for each hour
        # ** NEED TO ADJUST FOR INDOOR SETPOINT and MOUNTING LOCATION OF INDOOR UNITS 
        cop_interp = np.interp(dfh.db_temp, 
                               HomeEnergyModel.TEMPS_FIT, 
                               HomeEnergyModel.COPS_FIT)
        dfh['cop'] = cop_interp * self.hp_model.hspf / HomeEnergyModel.BASE_HSPF

        # adjustment to UA for insulation level.  My estimate, accounting
        # for better insulation *and* air-tightness as you move up the 
        # insulation scale.
        ua_insul_adj_arr = np.array([1.25, 1.0, 0.75])   # the adjustment factors by insulation level
        ua_insul_adj = ua_insul_adj_arr[s.insul_level - 1]   # pick the appropriate one
        
        # The UA values below are Btu/hr/deg-F
        # This is the UA / ft2 of the Level 2 (ua_insul_adj = 1) home
        # for the main living space.  Assume garage UA is about 10% higher
        # due to higher air leakage.
        # Determined this UA/ft2 below by modeling a typical Enstar home
        # and having the model estimate space heating use of about 1250 CCF.
        # See 'accessible_UA.ipynb'.
        ua_per_ft2 = 0.189
        ua_home = ua_per_ft2 * ua_insul_adj * s.bldg_floor_area * s.ua_true_up
        garage_area = (0, 14*22, 22*22, 36*25, 48*28)[s.garage_stall_count]
        ua_garage = ua_per_ft2 * 1.1 * ua_insul_adj * garage_area * s.ua_true_up

        # Balance Points of main home and garage
        # Assume a 10 deg F internal/solar heating effect for Level 2 insulation
        # in the main home and a 5 deg F heating effect in the garage.
        # Adjust the heating effect accordingly for other levels of insulation.
        htg_effect = np.array([10., 10., 10.]) / ua_insul_adj_arr
        balance_point_home = s.indoor_heat_setpoint - htg_effect[s.insul_level - 1]
        htg_effect = np.array([5.0, 5.0, 5.0]) / ua_insul_adj_arr  # fewer internal/solar in garage
        balance_point_garage = 55.0 - htg_effect[s.insul_level - 1]

        # BTU loads in the hour for the heat pump and for the secondary system.
        hp_load = []
        secondary_load = []

        # More complicated calculations are done in this hourly loop.  If processing
        # time becomes a problem, try to convert the calculations below into array
        # operations that can be done outside the loop.
        for h in dfh.itertuples():
            # calculate total heat load for the hour.
            # Really need to recognize that delta-T to outdoors is lower in the adjacent and remote spaces
            # if there heat pump is the only source of heat.
            home_load = max(0.0, balance_point_home - h.db_temp) * ua_home 
            garage_load = max(0.0, balance_point_garage - h.db_temp) * ua_garage
            total_load = home_load + garage_load
            if not h.running or s.no_heat_pump_use:
                hp_load.append(0.0)
                secondary_load.append(total_load)
            else:
                max_hp_output = s.hp_model.in_pwr_5F_max * h.cop * 3412.
                if s.hp_only:
                    hp_ld = min(home_load + garage_load * s.garage_heated_by_hp, max_hp_output)
                    hp_load.append(hp_ld)
                    secondary_load.append(total_load - hp_load)
                else:
                    # Nowhere near correct yet.  Just get a calc framework working.
                    hp_ld = min(home_load * s.pct_exposed_to_hp + garage_load * s.garage_heated_by_hp, max_hp_output)
                    hp_load.append(hp_ld)
                    secondary_load.append(total_load - hp_ld)

        dfh['hp_load_mmbtu'] = np.array(hp_load) / 1e6
        dfh['secondary_load_mmbtu'] = np.array(secondary_load) / 1e6

        # using array operations, calculate kWh use by the heat pump and 
        # the Btu use of secondary system.
        dfh['hp_kwh'] = dfh.hp_load_mmbtu / dfh.cop / 0.003412
        dfh['secondary_fuel_mmbtu'] = dfh.secondary_load_mmbtu / s.exist_heat_effic
        dfh['secondary_kwh'] = dfh.secondary_load_mmbtu * s.exist_kwh_per_mmbtu  # auxiliary electric use
        
        # Store annual and monthly totals.
        # Annual totals is a Pandas Series.
        total_cols = ['hp_load_mmbtu', 'secondary_load_mmbtu', 'hp_kwh', 'secondary_fuel_mmbtu', 'secondary_kwh']
        s.df_monthly = dfh.groupby('month')[total_cols].sum()
        dfm = s.df_monthly    # shortcut variable
        
        # Add in a column for the peak heat pump demand during the month
        dfm['hp_kw'] = dfh.groupby('month')[['hp_kwh']].max()
        
        # physical units for secondary fuel
        fuel = self.exist_heat_fuel
        dfm['secondary_fuel_units'] = dfm['secondary_fuel_mmbtu'] / fuel.btus * 1e6

        # COP by month
        dfm['cop'] = dfm.hp_load_mmbtu / (dfm.hp_kwh * 0.003412)   

        # Total kWh, heat pump + auxiliary of secondary system
        dfm['total_kwh'] = dfm.hp_kwh + dfm.secondary_kwh
                    
        # Total lbs of CO2 per month, counting electricity and fuel
        dfm['co2_lbs'] = dfm.total_kwh * s.co2_lbs_per_kwh + dfm.secondary_fuel_mmbtu * fuel.co2

        # Change index to Month labels
        s.df_monthly.index = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        
        s.ser_annual = s.df_monthly.sum().drop(['cop'])
        # Add the seasonal COP to the annual results
        tot = s.ser_annual
        if tot.hp_kwh:
            s.ser_annual['cop'] =  tot.hp_load_mmbtu * 1e6 / tot.hp_kwh / 3412.
        else:
            s.ser_annual['cop'] = np.nan
        
    def monthly_results(self):
        """Returns a DataFrame of monthly results.
        """
        return self.df_monthly
    
    def annual_results(self):
        return self.ser_annual
                
    def secondary_fuel_use(self):
        """Returns secondary fuel use in physical units, and also returns the
        units label.
        """
        fuel = self.exist_heat_fuel
        return self.ser_annual.secondary_fuel_units, fuel.unit
    
    def report(self):
        use, unit = self.secondary_fuel_use()
        s = f'Heat Pump: {self.ser_annual.hp_kwh:,.0f} kWh, Secondary Fuel Use: {use:,.2f} {unit}'
        s += f'\nSeasonal COP: {self.ser_annual.cop:.2f}'
        return s

## Main Controller Model that Runs Necessary Simulations to Estimate Impact of Heat Pump

In [65]:
"""Provides a class to model the impact of a heat pump on
energy use and cost.
"""
from os.path import join

this_dir = '/home/tabb99/heat-pump-study/general'
data_dir = join(this_dir, 'data')

def make_pattern(esc, life):
    """Makes a numpy array of length (life + 1) containing an escalation pattern
    that starts with a 1.0 in year 1 and escalates at the rate of 'esc' per year.
    """
    pat = np.ones(life - 1) * (1 + esc)
    return np.insert(pat.cumprod(), 0, [0.0, 1.0])

class HP_model:

    def __init__(self,
                 city_id,
                 utility_id,
                 co2_lbs_per_kwh,
                 exist_heat_fuel_id,
                 exist_unit_fuel_cost,
                 exist_fuel_use,
                 exist_heat_effic,
                 exist_kwh_per_mmbtu,
                 exist_is_point_source,
                 includes_dhw,
                 dhw_occupant_count,
                 includes_dryer,
                 elec_use_jan,
                 elec_use_may,
                 hp_model_id,
                 low_temp_cutoff,
                 garage_stall_count,
                 garage_heated_by_hp,
                 bldg_floor_area,
                 indoor_heat_setpoint,
                 insul_level,  
                 pct_exposed_to_hp,
                 doors_open_to_adjacent,
                 low_temp_acceptance,
                 capital_cost,
                 rebate_dol,
                 pct_financed,
                 loan_term,
                 loan_interest,
                 hp_life,
                 op_cost_chg,
                 sales_tax,
                 discount_rate,
                 inflation_rate,
                 fuel_esc_rate,
                 elec_esc_rate,
                ):

        # Store all of these input parameters as object attributes.
        args, _, _, values = inspect.getargvalues(inspect.currentframe())
        for arg in args[1:]:
            setattr(self, arg, values[arg])
            
        # Look up the objects associated with the IDs
        self.city = lib.city_from_id(city_id)
        self.utility = lib.util_from_id(utility_id)
        self.exist_fuel = lib.fuel_from_id(exist_heat_fuel_id)
        self.hp_model = lib.heat_pump_from_id(hp_model_id)
                    
        # Dummy results brought in below
        self.__df_monthly = pd.read_excel(join(data_dir, 'sample_monthly.xlsx'), index_col='month', sheet_name='with_pce')
        self.__df_monthly_no_pce = pd.read_excel(join(data_dir, 'sample_monthly.xlsx'), index_col='month', sheet_name='without_pce')

    def __repr__(self):
        """Returns a string with all the object attributes shown.  Text is truncated
        at 1,000 characters for attributes with long representations.
        """
        s = ''
        for attr in self.__dict__:
            val = repr(self.__dict__[attr])[:1000]
            if len(val)>70:
                s+=f'\n{attr}:\n{val}\n\n'
            else:
                s += f'{attr}: {val}\n'
        return s
        
    def check_inputs(self):
        pass
        
    def run(self):
        
        # shortcut for self
        s = self
        
        # shortcut to existing heating fuel
        fuel = s.exist_fuel

        # holds summary measures for the heat pump project (e.g. seasonal COP,
        # internal rate of return).
        s.summary = {}
        
        # Create the home energy simulation object
        sim = HomeEnergyModel(
            city_id=s.city_id,
            hp_model_id=s.hp_model_id,
            exist_heat_fuel_id=s.exist_heat_fuel_id,
            exist_heat_effic=s.exist_heat_effic,
            exist_kwh_per_mmbtu=s.exist_kwh_per_mmbtu,    
            exist_is_point_source=s.exist_is_point_source,
            co2_lbs_per_kwh=s.co2_lbs_per_kwh,
            low_temp_cutoff=s.low_temp_cutoff,
            garage_stall_count=s.garage_stall_count,
            garage_heated_by_hp=s.garage_heated_by_hp,
            bldg_floor_area=s.bldg_floor_area,
            indoor_heat_setpoint=s.indoor_heat_setpoint,
            insul_level=s.insul_level,            
            pct_exposed_to_hp=s.pct_exposed_to_hp,
            doors_open_to_adjacent=s.doors_open_to_adjacent,
            low_temp_acceptance=s.low_temp_acceptance,    
        )
        
        # Match the existing space heating use if it is provided.  Do so by using
        # the UA true up factor.
        # **** TO DO Deal with Electric Heat.  Calculate a space_fuel_use for electric
        # **** and then use the same calculation below.
        if s.exist_fuel_use is not None:
            
            # Remove DHW and Clothes dryer if they are present in the fuel use
            # number.
            space_fuel_use = s.exist_fuel_use
            if s.includes_dhw:
                dhw_use = s.dhw_occupant_count * 4.23e6 / fuel.dhw_effic / fuel.btus
                space_fuel_use -= dhw_use
                
            if s.includes_dryer:
                space_fuel_use -= 2.15e6 * s.dhw_occupant_count / fuel.btus
            
            sim.no_heat_pump_use = True
            sim.calculate()
            fuel_use1, _ = sim.secondary_fuel_use()
            
            # scale the UA linearly to attempt to match the target fuel use
            ua_true_up = space_fuel_use / fuel_use1
            sim.ua_true_up = ua_true_up
            sim.calculate()
            fuel_use2, _ = sim.secondary_fuel_use()
            
            # In case it wasn't linear, inter/extrapolate to the final ua_true_up
            slope = (fuel_use2 - fuel_use1)/(ua_true_up - 1.0)
            ua_true_up = 1.0 + (space_fuel_use - fuel_use1) / slope

        else:
            ua_true_up = 1.0
            
        print(f'UA True Up: {ua_true_up:.2f}')
        # Set the UA true up value into the model
        sim.ua_true_up = ua_true_up
        
        # Run the base case with no heat pump and record energy results
        sim.no_heat_pump_use = True
        sim.calculate()
        print(sim.secondary_fuel_use())
        s.df_mo_en_base = sim.monthly_results()
        
        # Run the model with the heat pump and record energy results
        sim.no_heat_pump_use = False
        sim.calculate()
        s.df_mo_en_hp = sim.monthly_results()
                
        # Create DataFrames that hold monthly energy cost amounts
        self.calc_monthly_cash()
        
        # Create a Cash Flow DataFrame and summary economic measures.
        self.calc_cash_flow()
            
    def calc_monthly_cash(self):
        """Calculates two DataFrames, s.df_mo_dol_base and s.df_mo_dol_hp, that contain
        the fuel and electricity costs in the base case (no heat pump) scenario and the
        with heat pump scenario.  A number of inputs found as object attributes are used. 
        """
        # shortcut to self
        s = self

        # The User supplied a January and a May kWh usage value that should
        # be used for the base case (no heat pump) total electricity use.
        # But, need to come up with a kWh value for every month.  Do that by
        # adjusting the kWh pattern available for this city.
        #
        # Determine the multiplier to adjust to the pattern to the actual.
        pat_use = np.array(s.city.avg_elec_usage)
        mult = (s.elec_use_jan - s.elec_use_may) / (pat_use[0] - pat_use[4])
        pat_use = mult * pat_use
        pat_use += s.elec_use_jan - pat_use[0]
        
        # Start the DataFrames, base and w/ heat pump
        # Each starts with just an index column with the month
        # Make shortcut variables as well.
        s.df_mo_dol_base = dfb = s.df_mo_en_base[[]].copy()
        s.df_mo_dol_hp = dfh = s.df_mo_en_base[[]].copy()
        
        # Make an object to calculate electric utility costs
        elec_cost_calc = ElecCostCalc(s.utility, sales_tax=s.sales_tax, pce_limit=500.0)
        # cost function that will be applied to each row of the cost DataFrame
        cost_func = lambda r: elec_cost_calc.monthly_cost(r.elec_kwh, r.elec_kw)

        # Now the no-PCE version
        util_no_pce = s.utility.copy()
        util_no_pce.PCE = 0.0    # blank out the PCE
        elec_cost_calc_no_pce = ElecCostCalc(util_no_pce, sales_tax=s.sales_tax, pce_limit=500.0)
        cost_func_no_pce = lambda r: elec_cost_calc_no_pce.monthly_cost(r.elec_kwh, r.elec_kw)

        dfb['elec_kwh'] =  pat_use
        # rough estimate of a base demand: not super critical, as the demand rate 
        # structure does not have blocks.  Assume a load factor of 0.4
        dfb['elec_kw'] = dfb.elec_kwh / 730.0 / 0.4
        dfb['elec_dol'] = dfb.apply(cost_func, axis=1)
        dfb['elec_dol_no_pce'] = dfb.apply(cost_func_no_pce, axis=1)

        # Now fuel
        dfb['secondary_fuel_units'] = s.df_mo_en_base.secondary_fuel_units
        dfb['secondary_fuel_dol'] = dfb.secondary_fuel_units * s.exist_unit_fuel_cost * (1. + s.sales_tax)

        # Total Electric + space heat
        dfb['total_dol'] =  dfb.elec_dol + dfb.secondary_fuel_dol
        dfb['total_dol_no_pce'] =  dfb.elec_dol_no_pce + dfb.secondary_fuel_dol

        # Now with the heat pump
        # determine extra kWh used in the heat pump scenario
        extra_kwh = (s.df_mo_en_hp.total_kwh - s.df_mo_en_base.total_kwh).values
        dfh['elec_kwh'] = dfb['elec_kwh'] + extra_kwh
        dfh['elec_kw'] =  dfb['elec_kw'] + s.df_mo_en_hp.hp_kw
        dfh['elec_dol'] = dfh.apply(cost_func, axis=1)
        dfh['elec_dol_no_pce'] = dfh.apply(cost_func_no_pce, axis=1)

        # Now fuel
        dfh['secondary_fuel_units'] = s.df_mo_en_hp.secondary_fuel_units
        dfh['secondary_fuel_dol'] = dfh.secondary_fuel_units * s.exist_unit_fuel_cost * (1. + s.sales_tax)

        # Total Electric + space heat
        dfh['total_dol'] =  dfh.elec_dol + dfh.secondary_fuel_dol
        dfh['total_dol_no_pce'] =  dfh.elec_dol_no_pce + dfh.secondary_fuel_dol
        
    def calc_cash_flow(self):
        """Calculates the cash flow impacts of the installation over the 
        life of the heat pump.  Creates a DataFrame, self.df_cash_flow, that shows
        the impacts. In that DataFrame, postive values are benefits and negative 
        values are costs. 
        Also calculates some summary economic measures that are added to
        the self.summary dictionary.
        """
        s = self   # shortcut variable

        # determine the changes caused by the heat pump on an annual basis.
        ann = (s.df_mo_dol_hp - s.df_mo_dol_base).sum()
        initial_cost = np.zeros(s.hp_life+1)
        
        initial_cost[0] = -s.capital_cost * (1 + s.sales_tax) * (1 - s.pct_financed) + s.rebate_dol
        loan_pmt = np.pmt(s.loan_interest, s.loan_term, s.capital_cost * (1 + s.sales_tax) * s.pct_financed)
        loan_cost = [0.0] + [loan_pmt] * s.loan_term + [0.0] * (s.hp_life -  s.loan_term)
        loan_cost = np.array(loan_cost)
        operating_cost = -s.op_cost_chg * make_pattern(s.inflation_rate, s.hp_life)
        fuel_cost = -ann.secondary_fuel_dol * make_pattern(s.fuel_esc_rate, s.hp_life)
        elec_cost = -ann.elec_dol * make_pattern(s.elec_esc_rate, s.hp_life)
        cash_flow = initial_cost + loan_cost + operating_cost + fuel_cost + elec_cost
        self.df_cash_flow = pd.DataFrame(
            {'initial_cost': initial_cost,
             'loan_cost': loan_cost,
             'operating_cost': operating_cost,
             'fuel_cost': fuel_cost,
             'elec_cost': elec_cost,
             'cash_flow': cash_flow
            }
        )
        self.df_cash_flow.index.name = 'year'
            
    def monthly_results(self, ignore_pce=False):
        """Returns a Pandas DataFrame of monthly results.
        If 'ignore_pce' is True, does not include PCE subsidy in analysis.
        """

        return self.__df_monthly_no_pce.copy() if ignore_pce else self.__df_monthly.copy()

    def annual_results(self, ignore_pce=False):
        """Returns a Pandas Series of annual results.
        If 'ignore_pce' is True, does not include PCE subsidy in analysis.
        """
        ann = self.monthly_results(ignore_pce=ignore_pce).sum()
        # need to fix the COP annual entry, which is not a simple sum
        # of the monthly values.
        hp_load = (self.__df_monthly.cop * self.__df_monthly.elec_chg_kwh).sum()
        ann_cop = hp_load / ann.elec_chg_kwh
        ann['cop'] = ann_cop
        return ann

    def cash_flow(self, ignore_pce=False):
        """Returns a Pandas DataFrame showing cash flow impacts over the
        life of the heat pump.
        If 'ignore_pce' is True, does not include PCE subsidy in analysis.
        """
        life = 14    # years of life
        ann = self.annual_results(ignore_pce=ignore_pce)
        capital = np.zeros(life+1)
        capital[0] = 4500.
        operating = 20. * make_pattern(.02, life)
        fuel_cost_savings = -ann.fuel_chg_dol * make_pattern(.04, life)
        elec_cost_increase = ann.elec_chg_dol * make_pattern(.03, life)
        cash_flow = -capital + -operating + fuel_cost_savings - elec_cost_increase
        df = pd.DataFrame(
            {'capital_cost': capital,
             'operating_cost': operating,
             'fuel_cost_savings': fuel_cost_savings,
             'elec_cost_increase': elec_cost_increase,
             'cash_flow': cash_flow
            }
        )
        df.index.name = 'year'

        return df

    def irr(self, ignore_pce=False):
        """Returns the internal rate of return of the heat pump investment.
        If 'ignore_pce' is True, does not include PCE subsidy in analysis.
        """
        df_cash = self.cash_flow(ignore_pce=ignore_pce)
        return np.irr(df_cash.cash_flow)

    def npv(self, ignore_pce=False):
        """Returns the net present value of the heat pump investment.
        If 'ignore_pce' is True, does not include PCE subsidy in analysis.
        """
        df_cash = self.cash_flow(ignore_pce=ignore_pce)
        return np.npv(0.05, df_cash.cash_flow)


In [55]:
inputs = dict(
    city_id=1,
    utility_id=1,
    co2_lbs_per_kwh=1.1,
    exist_heat_fuel_id=2,
    exist_unit_fuel_cost=0.97852,
    exist_fuel_use=1600,
    exist_heat_effic=.8,
    exist_kwh_per_mmbtu=8,
    exist_is_point_source=False,
    includes_dhw=True,
    dhw_occupant_count=3,
    includes_dryer=True,
    elec_use_jan=550,
    elec_use_may=400,
    hp_model_id=575,
    low_temp_cutoff=5,
    garage_stall_count=2,
    garage_heated_by_hp=False,
    bldg_floor_area=3600,
    indoor_heat_setpoint=70,
    insul_level=3,  
    pct_exposed_to_hp=0.46,
    doors_open_to_adjacent=False,
    low_temp_acceptance=2,
    capital_cost=4500,
    rebate_dol=500,
    pct_financed=0.5,
    loan_term=10,
    loan_interest=0.05,
    hp_life=14,
    op_cost_chg=10,
    sales_tax=0.02,
    discount_rate=0.05,
    inflation_rate=0.02,
    fuel_esc_rate=0.03,
    elec_esc_rate=0.02,
)

In [67]:
mc = HP_model(**inputs)
mc.run()
mc.df_mo_dol_base

UA True Up: 1.12
(1326.8147507731203, 'ccf')


,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,550.000000,1.883562,112.769671,112.769671,222.539988,222.115026,334.884697,334.884697
Feb,499.506846,1.710640,103.165903,103.165903,199.532713,199.151686,302.317589,302.317589
Mar,457.781650,1.567745,95.229796,95.229796,174.485553,174.152355,269.382151,269.382151
Apr,449.677284,1.539991,93.688350,93.688350,100.582838,100.390765,194.079115,194.079115
May,400.000000,1.369863,84.239760,84.239760,40.006817,39.930420,124.170180,124.170180
Jun,375.674221,1.286556,79.613012,79.613012,11.346484,11.324816,90.937828,90.937828
Jul,369.634753,1.265872,78.464309,78.464309,5.150210,5.140375,83.604684,83.604684
Aug,383.300240,1.312672,81.063476,81.063476,10.485142,10.465119,91.528595,91.528595
Sep,396.581210,1.358155,83.589509,83.589509,30.269908,30.212105,113.801613,113.801613
Oct,426.347723,1.460095,89.251082,89.251082,126.445891,126.204430,215.455511,215.455511


In [68]:
mc.df_mo_dol_hp

,elec_kwh,elec_kw,elec_dol,elec_dol_no_pce,secondary_fuel_units,secondary_fuel_dol,total_dol,total_dol_no_pce
Jan,900.714993,3.622227,179.475452,179.475452,179.421911,179.079286,358.554739,358.554739
Feb,978.920362,3.449306,194.350067,194.350067,142.961147,142.688148,337.038215,337.038215
Mar,1018.229926,3.261111,201.826722,201.826722,105.363748,105.162546,306.989268,306.989268
Apr,759.819242,2.396013,152.677165,152.677165,57.852995,57.742519,210.419684,210.419684
May,520.512737,1.944352,107.161211,107.161211,22.144534,22.102247,129.263458,129.263458
Jun,409.920531,1.718305,86.126640,86.126640,6.158736,6.146976,92.273615,92.273615
Jul,385.164874,1.564221,81.418128,81.418128,2.783759,2.778443,84.196571,84.196571
Aug,414.913445,1.734594,87.076289,87.076289,5.699625,5.688741,92.765030,92.765030
Sep,488.037274,2.052006,100.984397,100.984397,16.853586,16.821403,117.805800,117.805800
Oct,838.574323,3.004984,167.656334,167.656334,73.255359,73.115471,240.771805,240.771805


In [69]:
mc.df_cash_flow

,initial_cost,loan_cost,operating_cost,fuel_cost,elec_cost,cash_flow
year,,,,,,
0,-1795.0,0.000,-0.000000,0.000000,-0.000000,-1795.000000
1,0.0,-297.213,-10.000000,442.100298,-677.510665,-542.623366
2,0.0,-297.213,-10.200000,455.363307,-691.060879,-543.110571
3,0.0,-297.213,-10.404000,469.024207,-704.882096,-543.474889
4,0.0,-297.213,-10.612080,483.094933,-718.979738,-543.709885
5,0.0,-297.213,-10.824322,497.587781,-733.359333,-543.808873
6,0.0,-297.213,-11.040808,512.515414,-748.026520,-543.764913
7,0.0,-297.213,-11.261624,527.890877,-762.987050,-543.570797
8,0.0,-297.213,-11.486857,543.727603,-778.246791,-543.219044


In [18]:
mc.df_mo_en_base

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,0.0,18.461917,0.0,23.077397,147.695339,0.0,222.539988,NaN,147.695339,2862.520298
Feb,0.0,16.553234,0.0,20.691542,132.425871,0.0,199.532713,NaN,132.425871,2566.578916
Mar,0.0,14.475321,0.0,18.094152,115.802572,0.0,174.485553,NaN,115.802572,2244.398593
Apr,0.0,8.344352,0.0,10.430440,66.754818,0.0,100.582838,NaN,66.754818,1293.791816
May,0.0,3.318966,0.0,4.148707,26.551724,0.0,40.006817,NaN,26.551724,514.605606
Jun,0.0,0.941304,0.0,1.176630,7.530434,0.0,11.346484,NaN,7.530434,145.949228
Jul,0.0,0.427261,0.0,0.534077,3.418091,0.0,5.150210,NaN,3.418091,66.246881
Aug,0.0,0.869847,0.0,1.087309,6.958779,0.0,10.485142,NaN,6.958779,134.869832
Sep,0.0,2.511192,0.0,3.138989,20.089533,0.0,30.269908,NaN,20.089533,389.360252
Oct,0.0,10.489951,0.0,13.112439,83.919609,0.0,126.445891,NaN,83.919609,1626.466919


In [19]:
mc.df_mo_en_hp

,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,3.577076,14.884842,379.331599,18.606052,119.078734,1.738666,179.421911,2.763758,498.410333,2725.159464
Feb,4.693177,11.860057,516.958934,14.825071,94.880454,1.738666,142.961147,2.660737,611.839388,2407.556624
Mar,5.734345,8.740977,606.323036,10.926221,69.927813,1.693366,105.363748,2.771856,676.250848,2022.243756
Apr,3.544868,4.799484,338.500900,5.999356,38.395876,0.856022,57.852995,3.069242,376.896776,1116.511060
May,1.481855,1.837111,132.367577,2.296388,14.696884,0.574489,22.144534,3.281067,147.064462,430.448327
Jun,0.430376,0.510929,37.689314,0.638661,4.087430,0.431749,6.158736,3.346727,41.776744,120.677749
Jul,0.196321,0.230941,17.100687,0.288676,1.847525,0.298349,2.783759,3.364679,18.948212,54.618102
Aug,0.397006,0.472841,34.789257,0.591051,3.782727,0.421922,5.699625,3.344594,38.571984,111.582161
Sep,1.113018,1.398174,100.360208,1.747717,11.185388,0.693851,16.853586,3.250361,111.545596,327.183034
Oct,4.412686,6.077265,447.528093,7.596581,48.618117,1.544889,73.255359,2.889840,496.146209,1434.560779


In [20]:
mc.city

Name                                                         Anchorage
Latitude                                                        61.152
Longitude                                                     -149.864
ERHRegionID                                                          2
WAPRegionID                                                          2
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                         3.07
Oil2Price                                                          NaN
PropanePrice                                                       4.5
BirchPrice                                                         325
SprucePrice                                                        345
CoalPrice                                                          175
SteamPrice                                                         NaN
HotWat

In [21]:
# Model an average Enstar Home, probably somewhere between an
# insulation level 1 and 2.  Space Heating for Average Enstar
# House is about 1326 CCF, from "accessible_UA.ipynb"
m = HomeEnergyModel(**inputs)
m.insul_level = 2
m.exist_heat_effic = 0.76
m.bldg_floor_area = 2100
m.garage_stall_count = 1
m.no_heat_pump_use = True
m.calculate()
f_level2, _ = m.secondary_fuel_use()
print(m.report())

m.insul_level = 1
m.calculate()
f_level1, _ = m.secondary_fuel_use()
print(m.report())

# Assuming 2/3 Level 2 and 1/3 Level 1
print(0.67 * f_level2 + 0.33 * f_level1)

Heat Pump: 0 kWh, Secondary Fuel Use: 1,119.83 ccf
Seasonal COP: nan
Heat Pump: 0 kWh, Secondary Fuel Use: 1,508.89 ccf
Seasonal COP: nan
1248.221167662789


In [22]:
m.no_heat_pump_use = False
m.exist_heat_fuel_id = 2
m.calculate()
print(m.ser_annual)
m.df_monthly

hp_load_mmbtu              40.836879
secondary_load_mmbtu       78.081640
hp_kwh                   4218.306790
secondary_fuel_mmbtu      102.739000
secondary_kwh             624.653123
hp_kw                      13.215307
secondary_fuel_units      990.732887
total_kwh                4842.959913
co2_lbs                 17347.718953
cop                         2.837301
dtype: float64


,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh,hp_kw,secondary_fuel_units,cop,total_kwh,co2_lbs
Jan,3.672106,14.695524,388.306027,19.336216,117.564193,1.674816,186.463028,2.771610,505.870221,2818.794518
Feb,4.732318,11.750202,520.081329,15.460793,94.001619,1.674816,149.091539,2.666820,614.082948,2484.403973
Mar,5.873291,8.979712,619.617631,11.815410,71.837695,1.633998,113.938383,2.778106,691.455326,2143.003865
Apr,3.965370,5.412410,377.619781,7.121592,43.299282,0.871495,68.674952,3.077655,420.919064,1296.237289
May,2.167760,2.723327,192.829052,3.583325,21.786615,0.618300,34.554723,3.294806,214.615667,655.326233
Jun,0.999401,1.192362,87.266294,1.568898,9.538897,0.489936,15.129195,3.356481,96.805191,290.046723
Jul,0.685830,0.807605,59.698311,1.062639,6.460843,0.369970,10.247239,3.367018,66.159154,197.103796
Aug,0.975151,1.161668,85.159305,1.528510,9.293340,0.481098,14.739729,3.356066,94.452645,282.733569
Sep,1.765650,2.212536,157.729926,2.911232,17.700292,0.725641,28.073599,3.280814,175.430217,533.587403
Oct,4.750804,6.588195,478.813518,8.668677,52.705558,1.500210,83.593802,2.907982,531.519076,1598.906219


In [41]:
%%timeit
m.no_heat_pump_use = False
m.calculate()

283 ms ± 6.55 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
m

city_id: 1
hp_model_id: 575
exist_heat_fuel_id: 2
exist_heat_effic: 0.76
exist_kwh_per_mmbtu: 8
exist_is_point_source: False
co2_lbs_per_kwh: 1.1
low_temp_cutoff: 5
garage_stall_count: 1
garage_heated_by_hp: False
bldg_floor_area: 2100
indoor_heat_setpoint: 70
insul_level: 1
pct_exposed_to_hp: 0.46
doors_open_to_adjacent: False
low_temp_acceptance: 2
hp_only: False
no_heat_pump_use: False
ua_true_up: 1.0

city:
Name                                                         Anchorage
Latitude                                                        61.152
Longitude                                                     -149.864
ERHRegionID                                                          2
WAPRegionID                                                          2
FuelRefer                                                        False
FuelCityID                                                         NaN
Oil1Price                                                         3.07
Oil2Price        

In [25]:
m.df_hourly.head()

,db_temp,day_of_year,month,running,cop,hp_load_mmbtu,secondary_load_mmbtu,hp_kwh,secondary_fuel_mmbtu,secondary_kwh
timestamp,,,,,,,,,,
2018-01-01 00:30:00,21.02,1,1,True,2.674526,0.009352,0.013378,1.024861,0.017603,0.107028
2018-01-01 01:30:00,19.94,1,1,True,2.629053,0.009599,0.013754,1.070064,0.018098,0.110034
2018-01-01 02:30:00,21.02,1,1,True,2.674526,0.009352,0.013378,1.024861,0.017603,0.107028
2018-01-01 03:30:00,19.04,1,1,True,2.591158,0.009804,0.014067,1.108946,0.018510,0.112539
2018-01-01 04:30:00,19.04,1,1,True,2.591158,0.009804,0.014067,1.108946,0.018510,0.112539


In [26]:
df_mo = m.df_hourly.groupby('month')[['hp_kwh', 'secondary_fuel']].sum()
df_mo.hp_kwh.plot(kind='bar')

KeyError: "Columns not found: 'secondary_fuel'"

In [ ]:
# Check whether COP is getting modeled correctly
m.df_hourly.plot(x='db_temp', y='cop', marker='.', linewidth=0)